# 2 - Easy Wins (1 / 2)
##### **Author: Adam Gatt**

Later notebooks will cover more advance topics but the next two will aim to show off simple new language features that you should be able to immediately benefit from with little/no trade-off. Many revolve around enforcing correctness, to help remove some classes of bugs before they have a chance to happen. Additionally, they are largely compile-time features that leave no changes on the compiled code, resulting in no run-time cost.

The [clang-tidy](https://clang.llvm.org/extra/clang-tidy/) linter will be able to make suggestions about many of these language features, pointing out situation where they can be used. With some care and manual review, you can even run the linter over an entire existing codebase and have the suggestions applied automatically.

## override

How come the duck below fails to realise it can quack?

In [1]:
#include <iostream>

class Bird {
    public:
    virtual void makeNoise() const {
        std::cout << "Chirp!" << std::endl;
    }
};

class Duck : public Bird {
    public:
    virtual void makeNoise() {
        std:: cout << "Quack!" << std::endl;
    }
};

Bird* daffy = new Duck();

daffy->makeNoise();

Chirp!


It's because the two `makeNoise()` functions aren't the same! Because one is const and the other isn't, they count as having different signatures. This means the derived class isn't overriding the makeNoise in the base class; the two methods don't occupy the same entry in the vtable. 

Constness is a subtle way this bug can occur, but it can also be caused by a typo in the method name or differences in parameter types.

Because both the base and overridden methods use the same syntax (beginning `virtual` annotation), the compiler doesn't have a clear way to know that `Duck::makeNoise` is meant to be an overridden method instead of starting its own base virtual method. `override` is a new annotation to add to the overriding method and make it clear to the compiler. When a method is marked `override` the compiler will search for a matching base virtual method to ensure it exists, and will throw a compilation error if one cannot be found.

As an added benefit it also makes the intention clear to the human reader.

In [2]:
class Turkey : public Bird {
    public:
    virtual void makeNoise() override {
        std:: cout << "Gobble!" << std::endl;
    }
};

input_line_9:3:18: error: 'makeNoise' marked 'override' but does not override any member functions
    virtual void makeNoise() override {
                 ^


Interpreter Error: 

## nullptr

`nullptr` is a Modern C++ keywork for the null reference, where in the past you might have used `0` or `NULL` (often #defined to be 0). The advantage of nullptr is that it has its own type (`std::nullptr_t`), whereas the `int` type of NULL can allow for subtle bugs or ambiguities, especially in constructors and overloaded functions. In the best-case scenario the compiler will notice the ambiguity with a warning or error, in the worst-case you may have a subtle logic error.

In [3]:
class Colour {    
    public:
    int rgbCode;
    
    // Specify the rgb code directly
    Colour(int rgbCode) : rgbCode(rgbCode) { }
    
    // Copy the rgb code from another Colour
    Colour(const Colour* cloneSrc) {
        if (cloneSrc != NULL) {
            rgbCode = cloneSrc->rgbCode;
        }
    }
}

In [4]:
// Which constructor will be called?

Colour col1(NULL);

input_line_11:3:8: error: call to constructor of '__cling_N54::Colour' is ambiguous
Colour col1(NULL);
       ^    ~~~~
input_line_10:6:5: note: candidate constructor
    Colour(int rgbCode) : rgbCode(rgbCode) { }
    ^
input_line_10:9:5: note: candidate constructor
    Colour(const Colour* cloneSrc) {
    ^
input_line_10:1:7: note: candidate is the implicit copy constructor
class Colour {    
      ^
input_line_10:1:7: note: candidate is the implicit move constructor


Interpreter Error: 

In [5]:
Colour col2(nullptr);

Notice the constructor executing correctly instead of trying to dereference nullptr. This is because the guard clause `if (cloneSrc != NULL)` still performed its job. The new keyword enjoys strong interoperability with existing legacy code as the rules for comparing nullptr and NULL have been well-defined to produce intuitive results.

In [1]:
bool comparisons[] {
    nullptr == nullptr,
    nullptr != nullptr,
    nullptr == NULL,
    nullptr != NULL,
    nullptr == 0,
    nullptr != 0,
    nullptr ? true : false
};

comparisons

:13: warning: comparison between NULL and non-pointer ('nullptr_t' and NULL) [-Wnull-arithmetic]
    nullptr == NULL,
    ~~~~~~~ ^  ~~~~
input_line_7:6:13: warning: comparison between NULL and non-pointer ('nullptr_t' and NULL) [-Wnull-arithmetic]
    nullptr != NULL,
    ~~~~~~~ ^  ~~~~
input_line_7:9:5: warning: implicit conversion of nullptr constant to 'bool' [-Wnull-conversion]
    nullptr ? true : false
    ^~~~~~~ ~
    false


{ true, false, true, false, true, false, false }

Another example: when creating a nullable reference to a _Line_ object we accidentally leave out the asterisk and get unexpected results that will throw no errors. We would think that `NULL`, an int, shouldn't be assignable to a Line object. But despite the `=` character in the bottom-most line, the operation that actually takes place here is  initialisation rather than assignment. As such the constructor is called with the argument `NULL`, which the compiler happily treats as the int that it is.

In [2]:
#include <iostream>

class Line {
    private:
    int thickness;
    
    public:
    Line(int thickness) : thickness(thickness) {
        std::cout << "Created line with thickness " << thickness;
    }
};

Line myLine = NULL;

Created line with thickness 0

Wait, _what_? Who even asked for a constructor to be called here? More on this in the next notebook with `explicit`.

## Scoped enumerations

C++11 introduced "scoped" enumerations, an alternative enum syntax that can be simply achieved by adding just the one keyword "class" in the definition. Scoped enums are called so because their enum values (the "enumerants") are scoped to the enum itself rather than the class/namespace outside of it.

In [3]:
// Traditional enum declaration
enum Colour{
    RED,
    BLUE,
    GREEN
};

// New-style "scoped" enum declaration
enum class Flavour {
    VANILLA,
    CHOCOLATE,
    STRAWBERRY
};

Essentially this is a re-implementation of the enum concept with stricter rules to prevent potential errors. The new syntax is needed as previous-style enums must be left unchanged for compatibility purposes.
### Benefit: stronger type safety
Traditional "unscoped" enums map their values to some compiler-defined integral type indexed from 0, so the `GREEN` above will essentially be represented by the value 2. Traditional enums enforce no type checks when treated as (or implicitly converted to) an int. Type safety will be enforced in the other direction, however, so that the compiler won't implicitly cast an int to an enum value.

In [4]:
double calculateGST(double price) {
    return price * 1.1;
};

In [5]:
// Not even an compiler warning
calculateGST(Colour::GREEN)

2.2000000

In comparison, scoped enums are explicitly their own type and so this bug will be rejected by the compiler before it has a chance to occur.

In [6]:
calculateGST(Flavour::STRAWBERRY)

input_line_17:2:2: error: no matching function for call to 'calculateGST'
 calculateGST(Flavour::STRAWBERRY)
 ^~~~~~~~~~~~
input_line_14:1:8: note: candidate function not viable: no known conversion from 'Flavour' to 'double' for 1st argument
double calculateGST(double price) {
       ^


Interpreter Error: 

### Benefit: explicit choice of underlying data type 

If we want the enum to have an underlying type then we can specify it (as long as it is an integral type). We can then proceed to assign values to each individual enumerant as per usual.

In [7]:
enum class ChessPiece : short {
    PAWN = 1,
    KNIGHT = 3,
    ROOK = 5
};

In [8]:
short withCapture(short boardValue, ChessPiece piece) {
    return boardValue + static_cast<int>(piece);
}

withCapture(6, ChessPiece::ROOK)

11

### Benefit: tighter scope

Traditional enums are scoped to the environment surrounding the enum, so that `WRITE` below leaks its scope to `ConfigFile` and can be referenced with `ConfigFile::WRITE`. This allows the possibility of conflicts between different enums, like with the ambiguous `ConfigFile::READ`.

In [9]:
class ConfigFile {
    public:
    enum Operation{READ, WRITE};
    enum State{FOUND, OPENED, READ, FINALISED};
}

input_line_21:4:31: error: redefinition of enumerator 'READ'
    enum State{FOUND, OPENED, READ, FINALISED};
                              ^
input_line_21:3:20: note: previous definition is here
    enum Operation{READ, WRITE};
                   ^


Interpreter Error: 

Scoped enums do not leak their scope to the surrounding context. Here the two `READ` enums are `Operation::READ` and `State::READ` and cannot be confused. You can still use the class/namespace name to reference the enum value externally.  

In [10]:
class ConfigFile {
    public:
    enum class Operation{READ, WRITE};
    enum class State{FOUND, OPENED, READ, FINALISED};
};

ConfigFile::State::FOUND == ConfigFile::State::READ

false

In [11]:
// Same enum value name but strictly different values belonging to different enums.
ConfigFile::Operation::READ == ConfigFile::State::READ

input_line_24:3:29: warning: comparison of two values with different enumeration types ('__cling_N514::ConfigFile::Operation' and '__cling_N514::ConfigFile::State') [-Wenum-compare]
ConfigFile::Operation::READ == ConfigFile::State::READ
~~~~~~~~~~~~~~~~~~~~~~~~~~~ ^  ~~~~~~~~~~~~~~~~~~~~~~~
input_line_24:3:29: error: invalid operands to binary expression ('__cling_N514::ConfigFile::Operation' and '__cling_N514::ConfigFile::State')
ConfigFile::Operation::READ == ConfigFile::State::READ
~~~~~~~~~~~~~~~~~~~~~~~~~~~ ^  ~~~~~~~~~~~~~~~~~~~~~~~


Interpreter Error: 

An additional benefit comes from forcing external references to specify the full "path" to any enum value, and so the enum name itself is always there to provide context for the value. This is not the case for unscoped enums unless each enum value is named in a way that the context is unmistakeable. 

In [12]:
class Cat {
    public:
    enum EyeColour{GREEN, GREY, BLUE, BROWN};
};

// What did BROWN belong to? Is it the cat's fur colour, eye colour or something else?
int x = Cat::BROWN;

## Const references (pre-11)
Despite having introduced the useful `nullptr` earlier, the concept of the _null reference_ has been described as its creator as the [Billion Dollar Mistake](https://www.youtube.com/watch?v=YYkOWzrO3xg). Null-dereferencing errors are widespread and hidden, often only discovered by unexpected run-time errors. In C++, two sorts of variable references are available:

* C++ pointers (`*`) are "nullable references" and can be used to hold nullptr (or NULL/0)
* C++ references (`&`) are "non-nullable references" and cannot

References will never be null, and so they can never perform a null-dereference or be used before initialisation. You do not need a nullptr check before using them. Behind the scenes they compile as if they were pointers, resulting in no efficiency cost.

As such, if we are truly serious about preventing the Billion Dollar Mistake then:
> References should be our default choice, with pointers only for when nullability is a requirement or unavoidable.

In [13]:
#include <string>
void acceptStringRef(std::string& stringIn) {
    // We can do anything we want with stringIn inside this function with
    // complete confidence we will never cause a null-dereference error
};

Of course we can always call this function with a pointer that we de-reference first.

In [14]:
std::string* myString;
acceptStringRef(*myString)

input_line_28:3:18: warning: null passed to a callee that requires a non-null argument [-Wnonnull]
acceptStringRef(*myString)
                 ^~~~~~~~


Interpreter Exception: 

But still the function itself remains completely safe with the error occuring outside of it, in the `*myString` operation, closer to the site of the original pointer. The widespread use of references in function signatures help our reasoning during debugging, allowing us to eliminate "safe" parts of the codebase and preventing run-time exceptions from appearing deep within the call stack, far from the site of the actual logic error.

We can take this safety further with an important concept from Functional Programming, which is the idea of _immutability_. In FP, variables are "immutable" by default and this extends to input parameters in function definitions. If a function is guaranteed to never change a referenced input variable, then we can eliminate that function when debugging errors occuring with that variable outside the function. The variable had the same state both before and after the function was called, and the problem lies elsewhere.

Of course in C++ "immutable" _const_ variables are not the default and there are plenty of functions that are required to make changes to input parameters (e.g. std::fill). But the more functions that declare input references (and pointers) to be const, the easier it is to reason about the codebase when tracking down a bug.

And so, for further enforcing of correctness:
> const should be our default choice for references (and pointers), unless mutability is a requirement or unavoidable

And so we should look for all opportunities possible to convert our `type* name`s into `const type& name`s.

In [ ]:
// What does "processing" involve? Might this change our cart without our
// expecting it to? Will it break if cart is a nullptr?
void processOrder(Cart* cart);

// This is much safer in comparison
void processOrder(const Cart& cart);